In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install keras-tuner --upgrade

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import cv2

In [4]:
images_yes = [cv2.imread(file) for file in glob.glob("data/yes/*.*")]
images_no = [cv2.imread(file) for file in glob.glob("data/no/*.*")]
len(images_no)

98

In [33]:
def image_crop(image_path):
    cropped_image = []
    for i in image_path:
        if i is None:
            pass
        else:
            image = cv2.resize(i , (128, 128))
        
        #The initial processing of the image
        image = cv2.medianBlur(image, 3)
        image_bw = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        #The declaration of CLAHE 
        #clipLimit -> Threshold for contrast limiting
        clahe = cv2.createCLAHE(clipLimit = 5)
        final_img = clahe.apply(image_bw)
        cropped_image.append(final_img)
    cropped_image = np.array(cropped_image)
    return cropped_image

In [34]:
cropped_images_no = image_crop(images_no)
cropped_images_yes = image_crop(images_yes)

In [35]:
y_yes = np.ones(len(cropped_images_yes), dtype="int8")
y_no = np.zeros(len(cropped_images_no), dtype="int8")

In [36]:
X = np.concatenate((cropped_images_yes, cropped_images_no), axis=0)
y = np.concatenate((y_yes, y_no), axis=0)

# d1, d2, d3 = X.shape
# X = X.reshape((d1, d2 * d3))

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
from keras_tuner.engine.hyperparameters import HyperParameters
hp = HyperParameters()

In [39]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models

def build_model(hp):
    
    model = keras.models.Sequential()

    model.add(keras.layers.experimental.preprocessing.Rescaling(scale=1.0/255, input_shape=(128, 128)))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])

    return model

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 128, 128)          0         
                                                                 
 flatten (Flatten)           (None, 16384)             0         
                                                                 
 dense (Dense)               (None, 128)               2097280   
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
Total params: 2,115,082
Trainable params: 2,115,082
Non-trainable params: 0
_________________________________________________________________


In [40]:
import keras_tuner as kt

tuner = kt.BayesianOptimization(build_model, hyperparameters = hp, objective = 'val_loss')

INFO:tensorflow:Reloading Oracle from existing project ./untitled_project/oracle.json


In [41]:
tuner.search(X_train, y_train, validation_split = 0.2, batch_size = 32, epochs = 5)

best_model = tuner.get_best_models()[0]

Trial 2 Complete [00h 00m 02s]
val_loss: 2.4832985401153564

Best val_loss So Far: 2.4832985401153564
Total elapsed time: 00h 00m 02s
INFO:tensorflow:Oracle triggered exit


In [42]:
history = best_model.fit(X_train, y_train, epochs=5)

Epoch 1/5
7/7 [==============================] - 1s 17ms/step - loss: 1.4752 - accuracy: 0.7871
Epoch 2/5
7/7 [==============================] - 0s 20ms/step - loss: 1.3417 - accuracy: 0.7277
Epoch 3/5
7/7 [==============================] - 0s 22ms/step - loss: 0.7957 - accuracy: 0.8020
Epoch 4/5
7/7 [==============================] - 0s 19ms/step - loss: 0.3548 - accuracy: 0.8812
Epoch 5/5
7/7 [==============================] - 0s 21ms/step - loss: 0.4855 - accuracy: 0.8762


In [44]:
test_loss, test_acc = best_model.evaluate(X_test, y_test)
print(test_acc)

2/2 [==============================] - 0s 11ms/step - loss: 0.7228 - accuracy: 0.7647
0.7647058963775635
